In [15]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, subprocess
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
import nltk
from nltk.corpus import stopwords  
from nltk.stem import PorterStemmer
ps = PorterStemmer()
sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

#df, code = generate_code_from_db(ctx, qgroup=4028)

In [22]:
path=f'C:\dev\CyberScopeNextGen\CyberBalance.Scheduler\Server\Core\PowerShell\PowerShellScripts\GetCDMJsonData_FISMA_1.1.6.2_withtest.ps1'
o = subprocess.run(['powershell','-ExecutionPolicy', 'Unrestricted', '-command', path,"-IsDev", '$True'], capture_output=True, text=True)
o.stdout

'{\n                      "agency-dashboard-fmshrc":  {\n                                                                      "status":  "successful",\n                                                                      "indices":  "cdm_hwam_current",\n                                                                      "took":  1,\n                                                                      "timed_out":  false,\n                                                                      "_shards":  {\n                                                                                      "total":  1,\n                                                                                      "successful":  1,\n                                                                                      "skipped":  0,\n                                                                                      "failed":  0\n                                                                             

In [26]:
parse = json.loads(o.stdout)
buckets=parse['aggregations']['0']['buckets']
for b in buckets:
     agency=b['key']
     m=b['MODERATE']['1.1.6.2-MODERATE']['value']
     h=b['HIGH']['1.1.6.2-HIGH']['value']
     l=b['LOW']['1.1.6.2-LOW']['value']
     u=b['UNKNOWN']['1.1.6.2-UNKNOWN']['value']
     print(f'\tAssert.Equal("{agency}", item.Agency);')
     print(f'\tAssert.Equal("{m}", item.MODERATE);')
     print(f'\tAssert.Equal("{l}", item.LOW);')
     print(f'\tAssert.Equal("{h}", item.HIGH);')
     print(f'\tAssert.Equal("{u}", item.UNKNOWN);')
     print('}, item => {')


	Assert.Equal("DOL", item.Agency);
	Assert.Equal("0", item.MODERATE);
	Assert.Equal("0", item.LOW);
	Assert.Equal("0", item.HIGH);
	Assert.Equal("1", item.UNKNOWN);
}, item => {
	Assert.Equal("NRC", item.Agency);
	Assert.Equal("1", item.MODERATE);
	Assert.Equal("1", item.LOW);
	Assert.Equal("0", item.HIGH);
	Assert.Equal("0", item.UNKNOWN);
}, item => {
	Assert.Equal("TVA", item.Agency);
	Assert.Equal("0", item.MODERATE);
	Assert.Equal("0", item.LOW);
	Assert.Equal("1", item.HIGH);
	Assert.Equal("1", item.UNKNOWN);
}, item => {
	Assert.Equal("USAID", item.Agency);
	Assert.Equal("0", item.MODERATE);
	Assert.Equal("0", item.LOW);
	Assert.Equal("0", item.HIGH);
	Assert.Equal("1", item.UNKNOWN);
}, item => {


In [ ]:
df=sql_todf("""
SELECT EMAIL_CODE, EMAIL_DESCRIPTION, RecipientDescription FROM EMAIL_MASTER WHERE EMAIL_CODE LIKE '%23%'
""", ctx.connstr())
df['RecipientDescription']=df['RecipientDescription'].apply(lambda s: re.sub('<.*>','',s))
df
df.to_csv(r'c:\temp\emails.csv', index=False)



In [ ]:
lst = [
 'Subcomponent / Bureau / OpDiv'
,'System Name'
,'System Description'
,'FISMA Categorization YN'
,'System ID (Optional)'
,'High Value Asset ID (Optional)'
,'MFA Implemented? YN'
,'Does this system have sensitive data? YN'
,'Encryption of Sensitive Data at Rest (DAR) Implemented? YN'
,'Compensating Controls currently in place and operating effectively'
,'Reason for Non-Compliance'
,'Has a POA&M or set of POA&Ms been created to achieve compliance with these specific requirements? YN'
,'Expected Date to resolve MFA POA&M'
,'Expected Date to resolve DAR POA&M'
,'Expected Date to Resolve DIT POA&M' 
] 

In [ ]:
sql="""
SELECT COLUMN_NAME
  FROM INFORMATION_SCHEMA.COLUMNS
 WHERE TABLE_NAME LIKE '%2302' --AND (ORDINAL_POSITION > 9 AND ORDINAL_POSITION < 19)
ORDER BY ORDINAL_POSITION ASC
"""
df=sql_todf(sql, connstr)   
df

In [ ]:
frm="""
            DECLARE @COLUMN_NAME INT =  (SELECT COLUMN_NAME FROM BOD2302 WHERE PK_BOD2302=@PK_BOD2302) """
frm="""
			IF (SELECT COLUMN_NAME FROM BOD2302 WHERE PK_BOD2302=@PK_BOD2302) = NULL
			BEGIN
				INSERT INTO  @ErrorHolder 		
				SELECT 56500, 'Response is required for COLUMN_CAPTION row ' + @UniqueIdPoamId + ' ' AS Error
			END""" 

s='' 
for i,r in df.iterrows():
    s=s+frm.replace('COLUMN_NAME', r['COLUMN_NAME']).replace('COLUMN_CAPTION', r['COLUMN_CAPTION'])  
print(s)

In [ ]:
sheet=pd.read_excel(r'C:\Users\timko\Downloads\MFA.xlsx', sheet_name='CyberScope Inventory-Template', header=1, usecols='A:Z').reset_index()

In [ ]:
sheet_cols = sheet.columns[1:]
sheet_cols

In [ ]:
cols = []
for c in lst: 
    s=c.split(' ')
    s=' '.join([w for w in s if not w.lower() in sw])
    s=' '.join([s.title() for s in s.split(' ')])
    s=re.sub('[^A-Za-z\s]','',s) 
    s=re.sub('[^A-Za-z]','',s)
    s=re.sub('PoaM','POAM',s) 
    s=re.sub('Mfa','MFA',s) 
    s=re.sub('DataRestDarImplemented|OperatingEffectively|SpecificRequirements','',s) 
    r=f",{s} NVARCHAR(4000) NULL"
    if 'date' in s.lower(): 
        r=f",{s} DATETIME NULL"
    if 'yn' in s.lower(): 
        r=f",{s} INT NULL"
    r=re.sub('Yn','',r) 
    cols.append()
    print(r)

In [ ]:
ep="""EXECUTE sp_addextendedproperty @name = N'PicklistTypes.UsageType',@level0type = N'SCHEMA',@level0name = 'dbo',@level1type = N'Table', @level1name = 'BOD2302',@level2type = 'Column' 
,@level2name = 'COLUMN_NAME',@value = 'YN'; 
"""

ep="""EXECUTE sp_addextendedproperty @name = N'Control.Type',@level0type = N'SCHEMA',@level0name = 'dbo',@level1type = N'Table', @level1name = 'BOD2302',@level2type = 'Column' 
,@level2name = 'COLUMN_NAME',@value = 'RadTextBox'; 
"""


tmp="""const _COLUMN_NAME = $find($(`div[id$=_COLUMN_NAME]`).attr('id'));"""
tmp="""const _COLUMN_NAME = $find($(`div[id$=_COLUMN_NAME]`).attr('id'));"""


In [ ]:
df= 

In [ ]:
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df['DATA_TYPE']=df['DATA_TYPE'].str.upper()  
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].fillna(0).astype(int)
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].apply(lambda s: 4000 if s < 0 else s )
    df[cols] = df[cols].astype(str)
    df=df.loc[:, ['COLUMN_NAME','DATA_TYPE','CHARACTER_MAXIMUM_LENGTH' ]]
    return df

df=sql_todf("""
SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'MFAEncryption'  
""", connstr)   
cols2 = set(df.COLUMN_NAME)
df=normalizedf(df) 
df[-5:]
for i,r in df.iterrows():
    if not 'PK_' in r['COLUMN_NAME']: #  and 'INT' in r['DATA_TYPE'].upper():
        print(tmp.replace('COLUMN_NAME', r['COLUMN_NAME']))

In [ ]:
df=sql_todf("""
SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'MFAEncryption'  
""", connstr)    
COLUMN_NAME = df['COLUMN_NAME'][2:]   
DATE_TYPE = df['DATA_TYPE'][2:]   
cb=[]
for col in list ( zip( COLUMN_NAME  , DATE_TYPE, sheet_cols)  ) :
    if 'INT' in col[1]: 
        cb.append(f'<CB:DataField DBColumnName="{col[0]}" ImportColumnName="{col[2]}"  PickListTypeID="6"  runat="server"/>') 
    else:
        if 'Date' in col[0]:  
            cb.append(f'<CB:DataField DBColumnName="{col[0]}" Require="true" ImportColumnName="{col[2]}" DataType="System.DateTime"   runat="server"/>') 
        else: 
            cb.append(f'<CB:DataField DBColumnName="{col[0]}" Require="true" ImportColumnName="{col[2]}"  runat="server"/>') 
print( '\n'.join(cb) )


In [ ]:

def get_datafield(df):
    txt=[]   
    for i,r in df.iterrows(): 
        COLUMN_NAME=r['COLUMN_NAME'] 
        DATA_TYPE=r['DATA_TYPE']  
        s=f'<telerik:RadTextBox ID="{COLUMN_NAME}" runat="server"/>'
        if 'CHAR' not in DATA_TYPE:
            s=f'<telerik:RadComboBox ID="{COLUMN_NAME}" runat="server"/>'
        txt.append(f'{s}')
    txt='\n'.join(txt)  
    return txt

def get_sql(df,out=False):
    parms=[]
    for i,r in df.iterrows():
        s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()}({r['CHARACTER_MAXIMUM_LENGTH']}) = NULL"
        if r['DATA_TYPE'] == 'INT':
            s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL" 
        parms.append(f',{s}')

    into=', '.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c])
    vals=', @'.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c]) 
    ins=f'\n\n \t\t \t\t INSERT INTO MFAEncryption ({into})'
    ins=ins+f'\n \t\t \t\t VALUES (@{vals})'

    parms='\n\t\t'.join(parms)[:] 
    update=', '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])  
    ext='\t\t \t\t AND ' + ' AND '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])

    sel,exp=[],[]
    for i,r in df.iterrows(): 
        sel.append(f", {r['COLUMN_NAME']} ")
        if r['DATA_TYPE'] == 'INT':     
            sel.append(f", (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['COLUMN_NAME']}_Display]"  ) 
    sel='\t\t \t\t'+'\n \t\t \t\t'.join(sel)  
 
    return {'parms':parms,'ins':ins, 'update':update, 'sel':sel, 'ext':ext , 'exp':exp  }

def get_gridcols(df,out=False):
    ldf=df.to_dict(orient='records') 
    code,ddl,dv,sel,dtf=[],[],[],[],[]
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        t=f.read() 
        
        for d in ldf: 
            col=d['COLUMN_NAME'] 
            s=t.replace("{COLUMN_NAME}",d['COLUMN_NAME'])  
            if 'INT' in d['DATA_TYPE'].upper():
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}_Display")
                s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{col}" DataValueField="YN" runat="server" />')
                ddl.append(f'BindDDL(e, "{col}", "{col}", "{col}")')
                sel.append(f',(SELECT DisplayValue FROM vwPICKLISTS WHERE PK_Picklist={col}) AS {col}') 
            else: 
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}")
                s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{col}" runat="server" />')
                dv.append(f'DirectCast(e.Item.FindControl($"{col}"), RadTextBox).Text = drVal("{col}").ToString()')
                sel.append(f',{col}') 
            code.append(f'{s}')     
    code='\n'.join(code) 
    ddl='\n'.join(ddl)
    dv= '\n'.join(dv)
    sel= '\n'.join(sel)[1:]
    with open(f'{ctx.get_dest()}script.aspx', 'w', encoding='UTF-8') as fw:
        fw.write(f'{code} \n\n {ddl} \n\n {dv} \n\n {sel}')
    if out:  print(txt)
    return {'code':code, 'ddl':ddl,'dv':dv,'sel':sel, 'dtf': dtf}
  
 
p = get_gridcols(df, out=False)
p
print ( p['code'] ) 
#print (  get_sql( df )['ext']  )
#d=get_params(df, out=False)
#print (  get_params(df)['sel']  )

In [ ]:
print(  p['parms']  )

In [ ]:
sql = get_gridcols(df)
print( sql['code'])

In [ ]:
d=get_params(df, out=False)
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\MFAEncryption_CRUD.sql', 'r', encoding='UTF-8') as f:
    sql=f.read()
with open(r'CC:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\MFAEncryption_CRUD.sql', 'w', encoding='UTF-8') as f:
    sql=re.sub('--parms(?s).*--~parms',f'--parms\n{d["parms"]}\n\t\t--~parms', sql)
    sql=re.sub('--sel(?s).*--~sel',f'--sel\n{d["sel"]}\n\t\t--~sel', sql)
    sql=re.sub('--ins(?s).*--~ins',f'--ins\n{d["ins"]}\n\t\t--~ins', sql)
    sql=re.sub('--update(?s).*--~update',f'--update\n{d["update"]}\n\t\t--~update', sql)
    sql=re.sub('--ext(?s).*--~ext',f'--ext\n{d["ext"]}\n\t\t--~ext', sql)
    f.write(sql)
 

In [ ]:
d=get_params(df, out=False)
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\MFAEncryption_CRUD.sql', 'r', encoding='UTF-8') as f:
    sql=f.read()
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\MFAEncryption_CRUD.sql', 'w', encoding='UTF-8') as f:
    sql=re.sub('--parms(?s).*--~parms',f'--parms\n{d["parms"]}\n--~parms\n', sql)
    sql=re.sub('--sel(?s).*--~sel',f'--sel\n{d["sel"]}\n--~sel\n', sql)
    sql=re.sub('--ins(?s).*--~ins',f'--ins\n{d["ins"]}\n--~ins\n', sql)
    sql=re.sub('--update(?s).*--~update',f'--update\n{d["update"]}\n--~update\n', sql)
    f.write(sql)
#print(sql)
    #

In [ ]:
sheet=pd.read_excel(r'C:\Users\timko\Downloads\2302.xlsx', sheet_name='CyberScope Inventory Tab', header=0, usecols='A:G', nrows=3 ).reset_index(drop=True) 
lod=sheet.to_dict(orient='records')  

In [ ]:
import uuid 
rec=lod[0].copy()
rec['Unique ID']=  uuid.uuid4().hex
lod.append(rec)
lod


In [ ]:
pd.DataFrame(lod).to_excel(r'C:\Users\timko\Downloads\2302_new.xlsx', index = False)

In [ ]:
 
results=pd.DataFrame() 
engine = create_engine(ctx.connstr()) 
connection = engine.raw_connection()
cursor = connection.cursor()
try: 
    cursor.execute("BOD2302_CRUD @PK_OrgSubmission=?, @MODE=?, @UserMode=?, @UserId=?", ['147792', 'SELECT', 'Agency', '1838'])
    results =  list(cursor.fetchall())
    cursor.close()
    connection.commit()
finally:
    connection.close()
results